### TensorFlow and Keras-based implementation of a YOLO-inspired Convolutional Neural Network (CNN) for object detection, simplified to work on smaller datasets

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [2]:
# Constants
IMG_HEIGHT = 224
IMG_WIDTH = 224
IMG_CHANNELS = 3
GRID_SIZE = 7  # S x S grid
NUM_BOXES = 2  # B bounding boxes per grid cell
NUM_CLASSES = 11  # Change as per your dataset

In [3]:
# YOLO Output = S x S x (B*5 + C)
# 5 for each box: x, y, w, h, confidence
OUTPUT_CHANNELS = GRID_SIZE * GRID_SIZE * (NUM_BOXES * 5 + NUM_CLASSES)

In [4]:
def build_yolo_like_model(input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)):
    inputs = tf.keras.Input(shape=input_shape)

    # Feature extractor (CNN)
    x = layers.Conv2D(16, (3,3), strides=1, padding='same', activation='relu')(inputs)
    x = layers.MaxPooling2D((2,2))(x)

    x = layers.Conv2D(32, (3,3), strides=1, padding='same', activation='relu')(x)
    x = layers.MaxPooling2D((2,2))(x)

    x = layers.Conv2D(64, (3,3), strides=1, padding='same', activation='relu')(x)
    x = layers.MaxPooling2D((2,2))(x)

    x = layers.Conv2D(128, (3,3), strides=1, padding='same', activation='relu')(x)
    x = layers.MaxPooling2D((2,2))(x)

    x = layers.Conv2D(256, (3,3), strides=1, padding='same', activation='relu')(x)
    x = layers.Flatten()(x)

    x = layers.Dense(4096, activation='relu')(x)
    x = layers.Dropout(0.5)(x)

    # Final prediction layer
    outputs = layers.Dense(OUTPUT_CHANNELS, activation='sigmoid')(x)

    model = models.Model(inputs=inputs, outputs=outputs)
    return model


In [5]:
# Instantiate model
model = build_yolo_like_model()


In [6]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 224, 224, 16)        │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 112, 112, 16)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 112, 112, 32)        │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 56, 56, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 56, 56, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 28, 28, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 14, 14, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 50176)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 4096)                │     205,524,992 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 4096)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1029)                │       4,215,813 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 210,133,413 (801.60 MB)

 Trainable params: 210,133,413 (801.60 MB)

 Non-trainable params: 0 (0.00 B)